In this notebook, we will write our own k-means model from scratch and use it to cluster handwritten numbers from the MNIST dataset


In the cell below, we create an `assign_data` function, which takes the `data` and the `centers` for each cluster and makes an assignment of each datapoint in `data` to the closest of the `centers`, `centerids`.  We extract `n`, the number of datapoints, `d`, the dimensionality of the datapoints, and `k` the number of centers.

Next, we need to compute the squared distance between each center and each data point.

Reshaping the data to be 1 x `n` x `d`, and the centers to be `k` x 1 x `d` signals to numpy that when it subtracts these two arrays, it creates an array of shape `k` x `n` x `d`. That is, it computes all combinations of the `k` centers and the `n` datapoints for each of the `d` dimensions. We assign those differences to `res`.

Squaring each of the differences, then summing along dimension 2 --- that’s the components of the vectors --- produces the sum of squared distances, which is the squared distance between the centers and the datapoints.  The resulting array is of shape `k` x `n`.

`assign_data` also computes the `loss`, which the sum of the squared differences. We want to know which center has the *smallest* squared distance for each data point. `argmin` produces the index of an array with the smallest value along the given dimension. Here, we’re using dimension 0, which varies over the `k` centers. `centerids` is now an array with one integer for each datapoint that indicates which of the centers is closest.



In [ ]:
def assign_data(data,centers):
  '''For each datapoint in data, assign it to the closest center.
  args:
    data: ndarray of size n*d, n datapoints of d dimensions each.
    centers: ndarray of size k*d, k centers of same dimensions as datapoints.
  returns:
    centerids: ndarray of size n, indexes of centers assigned to each datapoint.
    loss: total distance from each datapoint to its assigned center.
  '''
  # n is the number of data points
  n = len(data)
  # d is the dimensionality of the data points
  d = len(data[0])
  # k is the number of clusters
  k = len(centers)
  # first, subtract the set of centers from each data point
  # Equivalent code: res = data[np.newaxis,:,:]-centers[:,np.newaxis,:]
  res = np.reshape(data,(1,n,d))-np.reshape(centers,(k,1,d))
  # sum the squared differences
  res2 = np.add.reduce(res**2,2)
  # assign each data point to its closest center
  centerids = np.apply_along_axis(np.argmin,0,res2)
  # While we're here, make a note of the loss
  loss = sum(np.apply_along_axis(np.min,0,res2))
  return(centerids, loss)

Next we'll compute the mean of each of the `k` centers using the `data` and their `centerids` assignments. `compute_means` takes the data and the center ids and computes the centers by averaging all of the datapoints with the same id.  This will be used to update the `centers`. 

After extracting the number of datapoints and dimension, we initialize the array of center locations to a `k` x `d` array of all zeros.

For each of the cluster id values from 0 to `k`, we do the following operations:
 - First, form a smaller array, `cols`, consisting of all the datapoints with the current center id.
 - To be robust, we make sure `cols` has a length greater than zero. That can happen if there’s a center that has been elbowed out of the running by the other centers being closer to all of the data points. 
  - If it equals zero, that means our center is out of the action and we should probably pick a different location for it. We simply choose one of the data points at random to be this new location.

- We want to move the center to the `mean` of the closest points. Numpy’s `mean` method computes the average of an array along any given dimension. Here, we choose dimension 0, which corresponds to the different data points. `mean` produces a component-wise average of all the data points with cluster id equal to `i`.
- After completing the loop, we return the newly computed `centers`.


In [ ]:
def compute_means(data, centerids, k):
  ''' For each cluster of datapoints, calculate its center.
  args:
    data: ndarray of size n*d, n datapoints of d dimentions each.
    centerids: ndarray of size n, the center ID assigned to each datapoint.
    k: int, number of centers.
  returns: ndarray of size k*d, k centers of d dimensions each.
  '''
  # n is number of data points
  n = len(data)
  # d is dimensionality of the data points
  d = len(data[0])
 
  # Zero out the centers
  centers = np.zeros(shape=(k,d))
 
  # loop over the clusters
  for i in range(k):
    # Gather the data points assigned to cluster i
    cols = np.array([data[j] for j in range(n) if centerids[j] == i])
    # Average to get mean for that cluster
    if len(cols) == 0: 
      centers[i] = data[random.randint(0,n-1)]
    else:
      centers[i] = cols.mean(0)
  return(centers)

With these two functions, we can build a `kmeans` model, which takes in the `data` and number of clusters, `k` and iteratively builds `k` clusters and updates them relative to the `loss`.  

We initialize the `k` centers by selecting random data points. We loop until the `loss` stops changing. If `oldloss` is different from the new `loss`, we use `assign_data` to assign each datapoint to its closest center. Then, we use `compute_means` to move the centers to the means of the points assigned to them. We repeat until the `loss` stops changing, returning the final `loss` and `centers`.  





In [ ]:
def kmeans(data, k):
  ''' Use k-means algorithm to divide data into k clusters.
  args:
    data: ndarray of size n*d, n datapoints of d dimensions each.
    k: int, number of clusters to create.
  returns:
    centers: ndarray of size k*d, k centers of d dimensions each.
    loss: total distance from each datapoint to its assigned center.
  '''
  n = len(data)
  d = len(data[0])
  # grab the centers from random points
  centers = data[[random.randint(0,n-1) for i in range(k)]]
  oldloss = 0
  loss = 1
  while oldloss != loss:
    oldloss = loss
    centerids, loss = assign_data(data,centers)
    centers = compute_means(data, centerids, k)
  return(centers, loss)

We will download the MNIST dataset and split the data into training data, `X_train` and `y_train` and test data, `X_test` and `y_test`.

In [ ]:
from sklearn.datasets import fetch_openml
data = fetch_openml(name='mnist_784')

import numpy as np
from sklearn.model_selection import train_test_split
# data.data is Pandas DataFrame of float64, data.target is a Series of
# categorical strings. We convert both into ndarray. We then convert
# the latter to integers, as stats.mode can't handle non-numerical values.
X = data.data.to_numpy()
y = np.array([int(i) for i in data.target])
_      , X_test, _      , y_test = train_test_split(X, y, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.33)
len(X_train)

Here, we run `kmeans` on our `X_train` data where `k=10`.  We run `kmeans` 9 times and record the `bestcenters` which have the `bestloss` among the recorded losses.  We then find the accuracy of these new centers on the test set.

In [ ]:
# (This will take a while.)

from scipy import stats
import math
from functools import reduce
import random

for nlabeled in [20]: # range(20,len(X_train),10):
  print(nlabeled)
  for k in [10]: # 2 # 5 # 20
    bestcenters, bestloss = kmeans(X_train, k)
    for rep in range(9):
      centers, loss = kmeans(X_train, k)
      if loss < bestloss: bestcenters, bestloss = centers, loss

    # How do we test the clustering that was discovered?
    # Assign testing points to clusters
    test_centerids, loss = assign_data(X_test, bestcenters)

    # Use the labeled examples to label the clusters
    train_centerids, loss = assign_data(X_train[:nlabeled], bestcenters)
    labs = y_train[:nlabeled]

    # Assign labels to clusters.
    # For each cluster, we find all labeled examples that belong to that cluster,
    # and then use the most common label among these examples. stats.mode randomly
    # selects one label if there is a tie.
    clust_labs = np.repeat(labs[0],k)
    for i in range(k):
      mode = stats.mode(labs[train_centerids == i]).mode
      if not np.isnan(mode):
        clust_labs[i] = mode

    # Print out the cluster labels. Ideally, each cluster would have a unique label,
    # but in reality we're likely to get multiple clusters with the same label.
    print(clust_labs)

    # Print accuracy score.
    print(sum(clust_labs[test_centerids] == y_test) / len(y_test))

  # What if we don't do any training at all, and just use the labeled examples
  # as cluster centers? This turned out to result in a better score.
  labids, loss = assign_data(X_test, X_train[:nlabeled])
  print(nlabeled, sum(y_train[labids] == y_test)/len(y_test))

In [ ]:
# The code above used nlabeled = 20, which was quite small.
# Try repeating with different nlabeled. The accuracy increases
# as nlabeled increases, but flats out around nlabeled=100.
for nlabeled in range(10, 201, 10):
  train_centerids, loss = assign_data(X_train[:nlabeled], bestcenters)
  labs = y_train[:nlabeled]
  clust_labs = np.repeat(labs[0],k)
  for i in range(k):
    mode = stats.mode(labs[train_centerids == i]).mode
    if not np.isnan(mode):
      clust_labs[i] = mode
  score = sum(clust_labs[test_centerids] == y_test) / len(y_test)

  labids, loss = assign_data(X_test, X_train[:nlabeled])
  score0 = sum(y_train[labids] == y_test) / len(y_test)

  print(nlabeled, score, score0)

We'll next print the images that best represent the centers of each of our clusters in K-means and the label for each of the clusters

We will also calcuate the percent accuracy of the clusters

In [ ]:
from keras.preprocessing.image import array_to_img

train_centerids, loss = assign_data(X_train, bestcenters)
test_centerids, loss = assign_data(X_test, bestcenters)

clust_labs = np.repeat(labs[0],k)
for i in range(len(bestcenters)):
  display(array_to_img(np.reshape(bestcenters[i],(28,28,1)), scale=False))
  clust_labs[i] = y_train[train_centerids == i][0]
  print(clust_labs[i])
#  mode = stats.mode(y_train[train_centerids == i]).mode
#  print(mode[0])
#  if len(mode) > 0: clust_labs[i] = mode[0]

sum(clust_labs[test_centerids] == y_test)/len(y_test)

Finally, we'll rewrite the K-means model as an active learning problem and perform semi-supervised clustering of the data

In [ ]:
from scipy import stats
import math
from functools import reduce

# ACTIVE LEARNING VERSION

# Compared to the earlier version, this cell has the following differences:
# * Different values of nlabeled and k (though nlabeled isn't actually used here).
# * Instead of pretending there are only a small number of labeled examples,
#   we have access to all training labels.
# * Out of all training datapoints belonging to a particular cluster, the label
#   of the first datapoint is used as the cluster label (instead of using the
#   most frequent label).
# An equivalent but more realistic scenario might be starting with unlabeled
# training data, do k-means clustering, and then take the first datapoint from
# each cluster for inspecting and labeling by a person.

for nlabeled in [10]: # range(20,len(X_train),10):
  for k in [50]: # 10 # 2 # 5 # 20 # range(10,200,50):
    bestcenters, bestloss = kmeans(X_train, k)
    for rep in range(9):
      centers, loss = kmeans(X_train, k)
      if loss < bestloss: bestcenters, bestloss = centers, loss
    # How do we test the clustering that was discovered?
    # Assign testing points to clusters
    test_centerids, loss = assign_data(X_test, bestcenters)

    # Let's label one example in each category
    train_centerids, loss = assign_data(X_train, bestcenters)

    clust_labs = np.repeat(labs[0],k)
    for i in range(len(bestcenters)):
      clust_labs[i] = y_train[train_centerids == i][0]
    print(clust_labs)

    # semi-supervised clustering
    print(k,sum(clust_labs[test_centerids] == y_test)/len(y_test))

    # nearest neighbors (from the first k training datapoints)
    labids, loss = assign_data(X_test, X_train[:k])
    print(k, sum(y_train[labids] == y_test)/len(y_test))